In [1]:
%matplotlib inline

import sys
import os
import numpy as np
import json
import pandas as pd
import traceback

sys.path.append("../../../") # add results folder as a python path 
from viz import *
from analyze import *

In [2]:
path = os.path.dirname("./data2/BOHB/iter100_result/")
target_dirs = os.listdir(path)

In [3]:
def getTrialResult(result):
    cum_exec_time = []
    cum_opt_time = []
    cum_first = True

    exec_times = []
    opt_times = []
    epochs = []
    model_info = []
    errors = []
    accuracies = []

    for r in result:
        dict_r = json.loads(r)
#         print(dict_r)
        exec_time = dict_r[2]['finished'] - dict_r[2]['started']
        opt_time = dict_r[2]['finished'] - dict_r[2]['submitted']

        if cum_first is True:
            cum_exec_time_last_value = 0.0
            cum_opt_time_last_value = 0.0
            cum_first = False

        if cum_exec_time_last_value <= 86400:
            exec_times.append(exec_time)
            opt_times.append(opt_time)
            cum_exec_time.append((cum_exec_time_last_value + exec_time))
            cum_opt_time.append((cum_opt_time_last_value + opt_time))
            epochs.append(dict_r[1])
            model_info.append(dict_r[0])
            if dict_r[3] is not None:
                errors.append(dict_r[3]['loss'])
                accuracies.append(dict_r[3]['info']['test accuracy'])
            else:
                errors.append(1.0)
                accuracies.append(0.0)
            
            cum_exec_time_last_value = cum_exec_time[len(cum_exec_time)-1]
            cum_opt_time_last_value = cum_opt_time[len(cum_opt_time)-1]
            
            dict_result = dict([("cum_exec_time", cum_exec_time),
                        ("exec_time", exec_times),
                        ("opt_time", opt_times),
                        ("model_info", model_info),
                        ("cum_opt_time", cum_opt_time),
                        ("error", errors),
                        ("accuracy", accuracies),
                        ("epoch", epochs)])
        else:
            continue
        
    return dict_result

In [4]:
def getTrialConfig(config):
    config_dict = dict()
    
    for c in config:
        dict_c = json.loads(c)
        target = dict_c[2]
        config_dict[str(dict_c[0])] = target
        
    return config_dict

In [5]:
def addModelInfo(trial_result, trial_config):
    model_index_list = []
    for i in range(len(trial_result['model_info'])):
        model_idx = str(trial_result['model_info'][i])
        model_info = trial_config[model_idx]
        target_index = -2
        if 'lookup_index' in model_info:
            target_index = model_info['lookup_index']
        elif trial_result['error'][i] == 1:
            target_index = 0
        model_index_list.append(target_index)
    trial_result['model_index'] = model_index_list
    del trial_result['model_info']

In [6]:
bohb_results = dict()
bohb_configs = dict()

for d in target_dirs:
# #     print(d)
#     if d != 'iter10_result':
#         continue
#     else:
#         trial_num = '0'
# #     trial_num = d[5:]
    trial_num = d[10:]
    print(trial_num)
    
    with open(path+'/'+d+'/'+'results.json') as rf:
        result = rf.readlines()

    result = [x.strip() for x in result]
    result_dict=getTrialResult(result)
    
    with open(path+'/'+d+'/'+'configs.json') as rf:
        config = rf.readlines()
        
    config = [x.strip() for x in config]
#     bohb_configs[trial_num]=getTrialConfig(config)
    config_dict=getTrialConfig(config)
    
    addModelInfo(result_dict, config_dict)
    
    bohb_results[trial_num]=result_dict
    

1-2
0-1
1-3
1-1
3-2
2-1
0-3
3-1
1-0
2-2
4-2
4-1
2-3
2-0
0-2
0-0


In [7]:
with open('iter100_result.json', 'w') as outfile:
    json.dump(bohb_results, outfile)

In [9]:
bohb_results['0-0']

{'cum_exec_time': [0.006141185760498047,
  0.010448694229125977,
  0.01481008529663086,
  0.018522024154663086,
  0.022316455841064453,
  0.026051759719848633,
  0.0298159122467041,
  0.03351902961730957,
  0.037148475646972656,
  0.042177438735961914,
  0.04715371131896973,
  0.05218386650085449,
  0.06264066696166992,
  0.06858038902282715,
  0.07389187812805176,
  0.0819544792175293,
  0.09037518501281738,
  0.09870672225952148,
  0.10681271553039551,
  0.11499404907226562,
  0.11902427673339844,
  0.12324404716491699,
  0.12767624855041504,
  0.13185572624206543,
  0.1362473964691162,
  0.1401076316833496,
  0.14453411102294922,
  0.14876341819763184,
  0.155320405960083,
  0.16028165817260742,
  0.16540312767028809,
  0.17055487632751465,
  0.17866015434265137,
  0.18369102478027344,
  0.18918156623840332,
  0.19462037086486816,
  0.20279884338378906,
  0.21087384223937988,
  0.21958446502685547,
  0.22803306579589844,
  0.2317333221435547,
  0.235579252243042,
  0.239551305770874